In [90]:
import os, shutil, pickle, shelve
#from Bio import SeqIO
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling1D, Flatten, Conv1D, LSTM, CuDNNLSTM, Bidirectional
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_normal
import utilities

In [4]:
def get_hybrid(opt):
    """  Return a hybrid network given a optimizer
    """
    model = Sequential()
    model.add(Conv1D(filters = 30, 
                     kernel_size = 15, 
                     padding = 'valid',
                     data_format = 'channels_last',
                     activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 5, strides = 5))
    model.add(Dropout(0.6))
    model.add(Bidirectional(LSTM(10)))
    model.add(Dropout(0.7))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
    return model

def train(model, train_x, train_y, val_data, config = {'epochs': 35, 'batch_size': 256}):
    """  Train model for a given config, training data, and validation data
    """
    epochs, batch_size = config['epochs'], config['batch_size']
    return model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=val_data)

In [34]:
def pad_for_detector(input_x, kernel_size):
    """ Pad the input matrix such that the (i, k) entry of the output 
        matrix is the score of motif detector k aligned to position i.
    input_x has shape = (N, n, 4)
    kernel_size has shape m
    output has shape = (N, n + 2m - 2, 4)
    """
    N, n, C = input_x.shape
    pad_value, num_pad = 0.25, kernel_size - 1
    pad_matrix = np.full((N, num_pad, C), pad_value)
    return np.concatenate((pad_matrix, input_x, pad_matrix), axis=1)
        
def get_pwm(test_seqs, kernel, bias):
    """
    test_seqs: (N, n + 2m - 2, 4)
    kernel: (m, 4)
    bias: (1, )
    output: (k, m, 4)
    """
    N, L, C = test_seqs.shape
    m, _ = kernel.shape
    
    ################################
    # Reconstruct the motif detector
    ################################
    conv_layer = Conv1D()
    
    ###############################################
    # Pass TEST_SEQUENCE through the motif detector
    # activation has a shape (N, n + m - 1)
    ###############################################
    activation = kernel(test_sequence)
    
    #################################################################
    # Extract those in the TEST_SEQUENCE that has least one position
    #     having positive activation.
    #################################################################
    
    # candidate and potential_start has a shape (N, )
    candidate, potential_start = np.max(activation, axis=1), np.argmax(activation, axis=1)
    
    # activated_indices should have a shape (k, )
    activated_indices = [i for i in range(N) if candidate[i] > 0]
    
    # activated_seq should have a shape(k, n + 2m - 2, 4)
    activated_seq, end = test_sequences[activated_indices, :, :], potential_start[activated_indices]
    start = end - m + 1
    activated_subseq = activated_seq[:, start:end+1, :]
    return activated_subseq

In [50]:
model = load_model('../../models/hybrid_net.h5')

filters = model.layers[0].get_weights()[0]
bias = model.layers[0].get_weights()[1]

In [63]:
filter0 = filters[:, :, 0]
bias0 = bias[0]
filter0.shape, bias0.shape

((15, 4), ())

In [65]:
N, n, m = 100, 1000, 15
test_seq = np.random.randint(2, size = N*n*4).reshape((N, n, 4))
padded_test_seq = pad_for_detector(test_seq, m)
padded_test_seq.shape

(100, 1028, 4)

In [66]:
filter0.shape, bias0.shape

((15, 4), ())

In [94]:
conv_layer = tf.layers.Conv1D(filters = 1,
                              kernel_size = m,
                              trainable=False)
weight, bias = tf.constant(filter0, dtype=tf.float32), tf.constant(bias0, dtype=tf.float32)